In [ ]:
!pip install music21

In [14]:
import os
import xml.etree.ElementTree as ET
import warnings
import music21


from music21 import *
from music21.musicxml.xmlObjects import MusicXMLImportException


class MusicXMLWarning(UserWarning):
    pass


class OttomanMusicPartParser(musicxml.xmlToM21.PartParser):
    def xmlMeasureToMeasure(self, mxMeasure: ET.Element) -> stream.Measure:        
        measureParser = OttomanMusicMeasureParser(mxMeasure, parent=self)
        # noinspection PyBroadException
        try:
            measureParser.parse()
        except MusicXMLImportException as e:
            e.measureNumber = str(measureParser.measureNumber)
            e.partName = self.stream.partName
            raise e
        except Exception as e:  # pylint: disable=broad-exception-caught
            warnings.warn(
                f'The following exception took place in m. {measureParser.measureNumber} in '
                + f'part {self.stream.partName}.',
                MusicXMLWarning
            )
            raise e

        self.lastMeasureParser = measureParser

        if measureParser.staves > self.maxStaves:
            self.maxStaves = measureParser.staves

        if measureParser.transposition is not None:
            self.updateTransposition(measureParser.transposition)

        self.firstMeasureParsed = True
        self.staffReferenceList.append(measureParser.staffReference)

        m = measureParser.stream
        self.setLastMeasureInfo(m)
        # TODO: move this into the measure parsing,
        #     because it should happen on a voice level.
        if measureParser.fullMeasureRest is True:
            # recurse is necessary because it could be in voices...
            r1 = m[note.Rest].first()

            if t.TYPE_CHECKING:
                # fullMeasureRest is True, means Rest will be found
                assert r1 is not None

            if self.lastTimeSignature is not None:
                lastTSQl = self.lastTimeSignature.barDuration.quarterLength
            else:
                lastTSQl = 4.0  # sensible default.

            if (r1.fullMeasure is True  # set by xml measure='yes'
                                    or (r1.duration.quarterLength != lastTSQl
                                        and r1.duration.type in ('whole', 'breve')
                                        and r1.duration.dots == 0
                                        and not r1.duration.tuplets)):
                r1.duration.quarterLength = lastTSQl
                r1.fullMeasure = True

        # NB: not coreInsert, because barDurationProportion()
        # is called in adjustTimeAttributesFromMeasure()
        self.stream.insert(self.lastMeasureOffset, m)
        self.adjustTimeAttributesFromMeasure(m)
        # TODO: musicxml4: listening

        return m


class OttomanMusicXMLImporter(musicxml.xmlToM21.MusicXMLImporter):
    """
    Subclass MusicXMLImporter to allow importing non-standard accidentals
    """
    def xmlPartToPart(self, mxPart, mxScorePart):
        '''
        Given a <part> object and the <score-part> object, parse a complete part.
        '''
        parser = OttomanMusicPartParser(mxPart, mxScorePart=mxScorePart, parent=self)
        parser.parse()
        if parser.appendToScoreAfterParse is True:
            return parser.stream
        else:
            return None


class OttomanMusicMeasureParser(musicxml.xmlToM21.MeasureParser):
    """
    Subclass MusicXMLImporter to allow importing non-standard accidentals
    """
    def nonTraditionalKeySignature(self, mxKey):
        '''
        Returns a KeySignature object that represents a nonTraditional Key Signature
        '''
        # noinspection PyShadowingNames
        allChildren = list(mxKey)

        lastTag = None
        allSteps = []
        allAlters = []
        allAccidentals = []

        for c in allChildren:
            tag = c.tag
            if lastTag == 'key-alter' and tag == 'key-step':
                allAccidentals.append(None)

            if tag == 'key-step':
                allSteps.append(c.text)
            elif tag == 'key-alter':
                allAlters.append(float(c.text))
            elif tag == 'key-accidental':
                allAccidentals.append(c.text)
            lastTag = tag

        if len(allAccidentals) < len(allAlters):
            allAccidentals.append(None)
        if len(allSteps) != len(allAlters):
            raise MusicXMLImportException(
                'For non traditional signatures each step must have an alter')

        ks = key.KeySignature(sharps=None)

        alteredPitches = []
        for i in range(len(allSteps)):
            thisStep = allSteps[i]
            thisAlter = allAlters[i]
            thisAccidental = allAccidentals[i]
            p = pitch.Pitch(thisStep)
            if thisAccidental is not None:
                if thisAccidental in self.mxAccidentalNameToM21:
                    accidentalName = self.mxAccidentalNameToM21[thisAccidental]
                else:
                    accidentalName = thisAccidental
                p.accidental = OttomanMusicAccidental(accidentalName)
                p.accidental.alter = thisAlter
            else:
                p.accidental = OttomanMusicAccidental(thisAlter)

            alteredPitches.append(p)

        ks.alteredPitches = alteredPitches
        return ks


class OttomanMusicAccidental(music21.pitch.Accidental):
    '''
    Subclass Accidental to allow importing non-standard accidentals
    '''
    def set(self, name, *args, allowNonStandardValue=True):
        # noinspection PyShadowingNames
        super().set(name, *args, allowNonStandardValue=allowNonStandardValue)


In [15]:
"""
Parse the Teslim and extract information relevant to inferring the makam.
- input: path to the musicxml for a sarki
- return: a dictionary of the following format:
    {
    "start_note": first note of the Teslim (music21 Note object),
    "end_note": last note of the Teslim (music21 Note),
    "range": (lowest note in the Teslim, highest note of the Teslim) (pair of music21 Notes),
    "accidentals": list of notes with accidentals in the order that they appear in the Teslim (list of music21 Notes)
    }
"""
folder_path = "../With Key Signatures/"

class DoesNotHaveTeslim(Exception):
    def __init__(self):
        self.message = "There does not exist a Teslim in this piece. "
        super().__init__(self.message)

def find_teslim(filename):
    MI = OttomanMusicXMLImporter()
    score = MI.scoreFromFile(folder_path+filename)
    start_measure_num = None
    end_measure_num = None

    for element in score.recurse().getElementsByClass(expressions.TextExpression):
        if element.content.lower() == 'teslim':
            start_measure_num =  element.getContextByClass('Measure').number
        if element.content.lower() == 'hane 2':
            end_measure_num = element.getContextByClass('Measure').number - 1
            break

    teslim = stream.Score()

    if start_measure_num is None:
        raise DoesNotHaveTeslim
    
    # Iterate through the measures in the original score and add them to the new score
    for measure in score.measures(start_measure_num, end_measure_num):
        teslim.insert(measure.offset, measure)

    return teslim

In [16]:
def extract_teslim_info(teslim):
    first_measure = teslim.parts[0].getElementsByClass('Measure')[0]
    start_note = first_measure.getElementsByClass('Note')[0]

    last_measure = teslim.parts[0].getElementsByClass('Measure')[-1]
    end_note = last_measure.getElementsByClass('Note')[-1]
    
    all_notes = teslim.parts[0].recurse().notes
    lowest_note = min(all_notes, key=lambda x: x.pitch)
    highest_note = max(all_notes, key=lambda x: x.pitch)
    
    accidentals = [note for note in all_notes if note.pitch.accidental is not None]
    
    result = {
        "start_note": start_note,
        "end_note": end_note,
        "range": (lowest_note, highest_note),
        "accidentals": accidentals
    }
    return result

In [17]:
folder_path = "../With Key Signatures/"

for musicfile in os.listdir(folder_path):
    try:
        teslim = find_teslim(musicfile)
    except Exception as e:
        print(musicfile)
        print(e)


CT 253 Vech‐i ‘Arazbâr Peşrevi. İsak Ağa’nın [Devr‐i Kebir].musicxml
There does not exist a Teslim in this piece. 
CT 256-257 Vech‐i ‘Arazbâr Sâz Semâ’îsi. İsak’ın.musicxml
There does not exist a Teslim in this piece. 
CT 76-80 İsak’ın İsfahân Peşrevi [Darb‐ı Fetih].musicxml
There does not exist a Teslim in this piece. 
